# Loading packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import gc
import keras_cv

from os import path
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model, Input, regularizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9048)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

1 Physical GPU, 1 Logical GPUs


In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 224
BATCH_SIZE = 512
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Loading datasets

In [5]:
labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'cifar-10', 'trainLabels.csv'))
labels_df.label = pd.Categorical(labels_df.label)
labels_df['codes'] = labels_df.label.cat.codes
labels_df['name'] = labels_df['id'].astype(str) + ".png"
labels_df.sort_values(by="name")
labels_df

,id,label,codes,name
0,1,frog,6,1.png
1,2,truck,9,2.png
2,3,truck,9,3.png
3,4,deer,4,4.png
4,5,automobile,1,5.png
...,...,...,...,...
49995,49996,bird,2,49996.png
49996,49997,frog,6,49997.png
49997,49998,truck,9,49998.png
49998,49999,automobile,1,49999.png


In [6]:
filenames = list(os.walk(path.join(os.getcwd(), '..', 'cifar-10', 'train', 'train_images'), topdown=False))[0][2]
filenames = pd.DataFrame(filenames, columns=["name"])
filenames = filenames.merge(labels_df, on="name")
filenames

,name,id,label,codes
0,1.png,1,frog,6
1,10.png,10,cat,3
2,100.png,100,automobile,1
3,1000.png,1000,dog,5
4,10000.png,10000,dog,5
...,...,...,...,...
49995,9995.png,9995,frog,6
49996,9996.png,9996,cat,3
49997,9997.png,9997,truck,9
49998,9998.png,9998,automobile,1


In [7]:
dataset_test = image_dataset_from_directory(
    directory=path.join(os.getcwd(), '..', 'cifar-10', 'test'),
    labels=None,
    label_mode=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=512
)
dataset_test = dataset_test.map(lambda x : preprocess_input(x)).prefetch(AUTOTUNE)

Found 300000 files belonging to 1 classes.


In [8]:
def get_model():
    resnet = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights="imagenet", include_top=False)

    for i, layer in enumerate(resnet.layers):
        if not layer.name.startswith('conv5'):
            layer.trainable = False

    x = layers.GlobalAveragePooling2D()(resnet.output)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(2048, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(N_CLASS, activation="softmax")(x)
    model = Model(resnet.input, output)
    return model

model = get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# Mix Up - augmentation

In [9]:
mix_up = keras_cv.layers.preprocessing.mix_up.MixUp(seed=SEED)

def apply_mix_up(x, y):
    samples = {"images": x, "labels": y}
    samples = mix_up(samples, training=True)
    return samples["images"], samples["labels"]

# Eksperymenty

In [10]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

In [11]:
os.makedirs("weights", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("results", exist_ok=True)

In [12]:
def get_train_valid_datasets(seed):
    dataset_train, dataset_val = image_dataset_from_directory(
        directory=path.join(os.getcwd(), '..', 'cifar-10', 'train'),
        labels=list(filenames['codes']),
        label_mode='int',
        validation_split=0.2,
        shuffle=True,
        subset='both',
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=512,
        seed=seed
    )
    dataset_train = dataset_train.map(lambda x, y : (x, tf.one_hot(y, N_CLASS)))
    dataset_train = dataset_train.map(apply_mix_up, num_parallel_calls=AUTOTUNE)
    dataset_train = dataset_train.map(lambda x, y: (preprocess_input(x), y))
    dataset_val = dataset_val.map(lambda x, y : (preprocess_input(x), tf.one_hot(y, N_CLASS))).prefetch(AUTOTUNE)
    return dataset_train, dataset_val

In [13]:
TRAINING_SEEDS = list(range(10))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = get_model()
    model_checkpoint = callbacks.ModelCheckpoint(
        f"weights/weights_{seed}.h5",
        monitor='val_categorical_accuracy',
        mode='max',
        save_best_only=True,
        save_weights_only=True,
        verbose=0
    )
    
    csv_logger = callbacks.CSVLogger(
        f"logs/{seed}.log", 
        separator=',', 
        append=False
    )
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy()]
    )
    
    dataset_train, dataset_val = get_train_valid_datasets(seed)

    history = model.fit(
        dataset_train,
        epochs=200,
        validation_data=dataset_val,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr, model_checkpoint, csv_logger],
        verbose=0
    )
    
    eval_results = model.predict(dataset_test)
    np.save(f"results/result_{seed}", eval_results)
    gc.collect()


Found 50000 files belonging to 1 classes.
Using 40000 files for training.
Using 10000 files for validation.

Epoch 19: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 30: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 40: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.

Epoch 45: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
586/586 [==============================] - 282s 474ms/step
Found 50000 files belonging to 1 classes.
Using 40000 files for training.
Using 10000 files for validation.

Epoch 19: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 31: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 36: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
586/586 [==============================] - 281s 473ms/step
Found 50000 files belonging to 1 classes.
Using 40000 files for training.
Using 10000 files for validation.

Epoch 21: Re